In [12]:
%cd '/content/drive/MyDrive/Codeforce-ratings-prediction'

/content/drive/MyDrive/Codeforce-ratings-prediction


# Importing Libraries

In [13]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff
pd.options.plotting.backend = "matplotlib"
import category_encoders as ce
from statsmodels.stats.outliers_influence import variance_inflation_factor

import textwrap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, SGDRegressor
from sklearn.metrics import r2_score, mean_absolute_error

# Dataset Loaded

In [14]:
df = pd.read_csv('data.csv')
df.head()

,userid,rating,contest1,contest2,contest3,contest4,contest5,contest6,contest7,contest8,contest9,contest10
0,3143927301,2115,2078,2055,2115,2047.0,2024.0,2010.0,1953.0,1936.0,2042.0,2045.0
1,1876577621,2254,2194,2114,2152,2179.0,2211.0,2154.0,2170.0,2141.0,2157.0,2209.0
2,6397741793,2344,2120,2206,2147,2234.0,2294.0,2090.0,2089.0,2072.0,2085.0,2114.0
3,3090123616,2224,2224,2222,2166,2116.0,2029.0,2113.0,2104.0,2096.0,2115.0,2163.0
4,9564162806,2128,2128,2120,2072,2018.0,1963.0,2039.0,1932.0,1963.0,1960.0,1886.0


# Basic Data Exploration

In [15]:
df.shape

(1423, 12)

In [16]:
df.dtypes

userid         int64
rating         int64
contest1       int64
contest2       int64
contest3       int64
contest4     float64
contest5     float64
contest6     float64
contest7     float64
contest8     float64
contest9     float64
contest10    float64
dtype: object

In [17]:
df.isnull().sum()

userid         0
rating         0
contest1       0
contest2       0
contest3       0
contest4       1
contest5      19
contest6      49
contest7      80
contest8     120
contest9     144
contest10    171
dtype: int64

***There are lots of missing values specially from contest 7 to 10.***

In [19]:
df.duplicated().sum()

0

***There are no duplicated values.***

In [21]:
df.drop(columns=['userid'], inplace=True)

In [23]:
df.describe()

,rating,contest1,contest2,contest3,contest4,contest5,contest6,contest7,contest8,contest9,contest10
count,1423.000000,1423.000000,1423.000000,1423.000000,1422.000000,1404.000000,1374.000000,1343.000000,1303.000000,1279.000000,1252.000000
mean,2291.403373,2225.569220,2182.663387,2144.553760,2096.829114,2062.668803,2044.155022,2032.603127,2032.438987,2018.510555,2006.584665
std,258.730165,228.588617,247.831336,277.342857,324.225868,358.430332,375.658912,388.340663,374.310185,385.680233,397.971750
min,1054.000000,1029.000000,1001.000000,838.000000,683.000000,570.000000,427.000000,439.000000,388.000000,453.000000,354.000000
25%,2121.000000,2077.000000,2022.500000,1984.000000,1940.250000,1913.750000,1894.250000,1884.500000,1877.500000,1855.000000,1843.750000
50%,2224.000000,2155.000000,2121.000000,2103.000000,2080.000000,2069.500000,2045.500000,2044.000000,2041.000000,2033.000000,2014.000000
75%,2396.500000,2314.000000,2288.000000,2278.500000,2248.000000,2246.250000,2231.750000,2221.500000,2213.000000,2201.500000,2205.250000
max,3833.000000,3572.000000,3697.000000,3569.000000,3487.000000,3644.000000,3833.000000,3724.000000,3727.000000,3783.000000,3813.000000


# Data Viz

In [27]:
fig = make_subplots(rows=3, cols=4, subplot_titles=df.columns)

fig.add_trace(go.Histogram(x=df['rating'], name='rating'), row=1, col=1)
fig.add_trace(go.Histogram(x=df['contest1'],  name='contest1'), row=1, col=2)
fig.add_trace(go.Histogram(x=df['contest2'], name='contest2'), row=1, col=3)
fig.add_trace(go.Histogram(x=df['contest3'], name='contest3'), row=1, col=4)
fig.add_trace(go.Histogram(x=df['contest4'], name='contest4'), row=2, col=1)
fig.add_trace(go.Histogram(x=df['contest5'], name='contest5'), row=2, col=2)
fig.add_trace(go.Histogram(x=df['contest6'], name='contest6'), row=2, col=3)
fig.add_trace(go.Histogram(x=df['contest7'], name='contest7'), row=2, col=4)
fig.add_trace(go.Histogram(x=df['contest8'], name='contest8'), row=3, col=1)
fig.add_trace(go.Histogram(x=df['contest9'], name='contest9'), row=3, col=2)
fig.add_trace(go.Histogram(x=df['contest10'], name='contest10'), row=3, col=3)

# Update layout
fig.update_layout(title='Individual Displots of Eleven Columns', showlegend=False)
fig.update_xaxes(title_text='Value', row=3, col=2)
fig.update_yaxes(title_text='Probability Density', row=2, col=1)

# Show plot
fig.show()


In [28]:
fig = make_subplots(rows=3, cols=4, subplot_titles=df.columns)

fig.add_trace(go.Box(x=df['rating'], name='rating'), row=1, col=1)
fig.add_trace(go.Box(x=df['contest1'],  name='contest1'), row=1, col=2)
fig.add_trace(go.Box(x=df['contest2'], name='contest2'), row=1, col=3)
fig.add_trace(go.Box(x=df['contest3'], name='contest3'), row=1, col=4)
fig.add_trace(go.Box(x=df['contest4'], name='contest4'), row=2, col=1)
fig.add_trace(go.Box(x=df['contest5'], name='contest5'), row=2, col=2)
fig.add_trace(go.Box(x=df['contest6'], name='contest6'), row=2, col=3)
fig.add_trace(go.Box(x=df['contest7'], name='contest7'), row=2, col=4)
fig.add_trace(go.Box(x=df['contest8'], name='contest8'), row=3, col=1)
fig.add_trace(go.Box(x=df['contest9'], name='contest9'), row=3, col=2)
fig.add_trace(go.Box(x=df['contest10'], name='contest10'), row=3, col=3)

# Update layout
fig.update_layout(title='Individual Box Plots of Eleven Columns', showlegend=False)

# Rotate y-axis labels
fig.update_yaxes(tickangle=90)

# Show plot
fig.show()


In [38]:
fig = px.scatter_matrix(df)
fig.update_traces(diagonal_visible=False)
fig.update_layout(height=900)
fig.update_traces(diagonal_visible=False)
fig.show()